In [1]:
import sys
sys.path.append('pytorchvideo')
from pytorchvideo.data import LabeledVideoDataset


In [2]:
from glob import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt



In [3]:
nvlc = glob("D:/Sun'iyX/ML/Datasets/violence/Real Life Violence Dataset/NonViolence/*")
vlc = glob("D:/Sun'iyX/ML/Datasets/violence/Real Life Violence Dataset/Violence/*")

label = [0]*len(nvlc)+[1]*len(vlc)

df = pd.DataFrame(zip(nvlc+vlc, label), columns = ['file', 'label'])
print('non violance video ', len(nvlc))
print('violance video ', len(vlc))
df.head()




non violance video  1000
violance video  1000


,file,label
0,D:/Sun'iyX/ML/Datasets/violence/Real Life Viol...,0
1,D:/Sun'iyX/ML/Datasets/violence/Real Life Viol...,0
2,D:/Sun'iyX/ML/Datasets/violence/Real Life Viol...,0
3,D:/Sun'iyX/ML/Datasets/violence/Real Life Viol...,0
4,D:/Sun'iyX/ML/Datasets/violence/Real Life Viol...,0


In [4]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=0.2, shuffle = True)
len(train_df), len(val_df)

(1600, 400)

In [5]:
train_df.head()

,file,label
1712,D:/Sun'iyX/ML/Datasets/violence/Real Life Viol...,1
845,D:/Sun'iyX/ML/Datasets/violence/Real Life Viol...,0
268,D:/Sun'iyX/ML/Datasets/violence/Real Life Viol...,0
1157,D:/Sun'iyX/ML/Datasets/violence/Real Life Viol...,1
1843,D:/Sun'iyX/ML/Datasets/violence/Real Life Viol...,1


Augmentation


In [6]:
from pytorchvideo.data import LabeledVideoDataset, make_clip_sampler, labeled_video_dataset

from pytorchvideo.transforms import (
    ApplyTransformToKey,
    Normalize,
    RandomShortSideScale,
    
    UniformTemporalSubsample,
    Permute
)

from  torchvision.transforms import (
    Compose,
    Lambda,
    RandomCrop,
    RandomHorizontalFlip,
    Resize

)

from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo
)

c:\Users\USER1\.virtualenvs\ML-qnch7FHx\Lib\site-packages\torchvision\transforms\functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(
c:\Users\USER1\.virtualenvs\ML-qnch7FHx\Lib\site-packages\torchvision\transforms\_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
c:\Users\USER1\.virtualenvs\ML-qnch7FHx\Lib\site-packages\torchvision\transforms\_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.w

In [7]:
video_trasnform = Compose([
    ApplyTransformToKey(key='video',
    transform=Compose([
        UniformTemporalSubsample(20),
        Lambda(lambda x:x/255),
        Normalize((0.45,0.45,0.45),(0.225,0.225,0.225)),
        RandomShortSideScale(min_size=248, max_size=256),
        CenterCropVideo(224),
        RandomHorizontalFlip(p=0.5)
    ])
    )
])

In [8]:
from torch.utils.data import DataLoader
train_dataset = labeled_video_dataset( data_path="D:/Sun'iyX/ML/Datasets/violence/Real Life Violence Dataset/",
                                      clip_sampler=make_clip_sampler('random',2),
                                      transform=video_trasnform,decode_audio=False # fro the smart tv project it will be true

                                      )

loader = DataLoader(train_dataset, batch_size= 5, num_workers = 0, pin_memory= True)


In [9]:
batch = next(iter(loader))

In [10]:
batch.keys()

dict_keys(['video', 'video_name', 'video_index', 'clip_index', 'aug_index', 'label'])

In [11]:
batch['video'].shape, batch['label'].shape

(torch.Size([5, 3, 20, 224, 224]), torch.Size([5]))

Model Architecture

In [12]:
import torch.nn as nn 
import torch
from pytorch_lightning import LightningModule, seed_everything, Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor
from torch.optim.lr_scheduler import CosineAnnealingLR
from sklearn.metrics import classification_report
import torchmetrics


In [13]:
class OurModel(LightningModule):
    def __init__(self):
        super(OurModel,self).__init__()
        # model architechture
        self.video_model = torch.hub.load('facebookresearch/pytorchvideo','efficient_x3d_xs', pretrained = True)
        self.relu = nn.ReLU()
        self.linear = nn.Linear(400, 1)

        self.lr = 0.001
        self.batch_size = 4
        self.numworker = 4

        # evaluation metric

        self.metric =  torchmetrics.Accuracy(task="binary")
        # loss funtion
        self.criterion = nn.BCEWithLogitsLoss()

        def forward(self, x):
            x = self.video_model(x)
            x = self.relu(x)
            x=self.linear(x)
            return x
        
        def configure_optimizers(self):
            opt = torch.optim.AdamW(params=self.parameters(), lr = self.lr)
            scheduler = CosineAnnealingLR(opt, T_max = 10, eta_min = le-6, last_epoch = -1)
            return {'optimizer': opt, 'lr_scheduler': scheduler}
        
        def train_dataloader(self):
            dataset = labeled_video_dataset( 
                train_df, 
                clip_sampler= make_clip_sampler('random', 2),
                transform= video_trasnform, decode_audio=False
            )
        
            loader = DataLoader(dataset, batch_size = self.batch_size, num_workers = self.numworker, pin_memory=True)
            return loader
        def train_step(self, btach, batch_idx):
            video, label = batch['video'], batch['label']
            out = self(video)
            loss = self.criterion(out, label)
            metric = self.metric(out, label.to(torch.int64))
            return {'loss': loss, 'mertric': metric.detach()}
        
        def train_epoch_end(self, outputs):
            loss = torch.stack([x['loss'] for x in outputs]).mean().cpu().numpy().round(2)
            metric = torch.stack([x['metric'] for x in outputs]).mean().cpu().numpy().round(2)
            self.log('training_loss', loss)
            self.log('training metric',metric)

        def val_dataloader(self):
            dataset = labeled_video_dataset( 
                val_df, 
                clip_sampler= make_clip_sampler('random', 2),
                transform= video_trasnform, decode_audio=False
            )
        
            loader = DataLoader(dataset, batch_size = self.batch_size, num_workers = self.numworker, pin_memory=True)
            return loader
        
        def validation_step(self, batch, batch_idx):
            video, label = batch['video'], batch['label']
            out = self(video)
            loss = self.criterion(out, label)
            metric = self.metric(out, label.to(torch.int64))
            return {'loss': loss, 'mertric': metric.detach()}
        
        def validation_epoch_end(self, outputs):
            loss = torch.stack([x['loss'] for x in outputs]).mean().cpu().numpy().round(2)
            metric = torch.stack([x['metric'] for x in outputs]).mean().cpu().numpy().round(2)
            self.log('validation_loss', loss)
            self.log('validation_metric',metric)

        def test_dataloader(self):
            dataset = labeled_video_dataset( 
                val_df, 
                clip_sampler= make_clip_sampler('random', 2),
                transform= video_trasnform, decode_audio=False
            )
        
            loader = DataLoader(dataset, batch_size = self.batch_size, num_workers = self.numworker, pin_memory=True)
            return loader
        
        def test_step(self):
            video, label = batch['video'], batch['label']
            out = self(video)
            return {'label': label.detach(), 'pred': out.detach()}
        
        def test_epoch_end(self, outputs):
            label = torch.cat([x['label'] for x in outputs]).cpu().numpy()
            pred = torch.cat([x['label'] for x in outputs]).cpu().numpy()
            pred = np.where(pred>0.5, 1, 0)
            print(classification_report(label, pred))

            


        


In [14]:
chekpoint_callback = ModelCheckpoint(
    monitor = 'val_loss',
    dirpath = 'chekpoints',
    filename = 'file', 
    save_last = True
)
lr_monitor = LearningRateMonitor(logging_interval='epoch')

In [21]:
from ray import tune

tuner = tune.Tuner(
            tune.with_resources(train_model, {'cpu':10, 'gpu': 1}),
            tune_config=tune.TuneConfig(
                metric="loss",
                mode="min",
                num_samples=num_samples,
                scheduler=scheduler,
                max_concurrent_trials=max_concurrent_trials,
            ),
            param_space=config,
            run_config=air.RunConfig(name="tune"),
        )

c:\Users\USER1\.virtualenvs\ML-qnch7FHx\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-12-29 08:33:51,679	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-12-29 08:33:52,619	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


NameError: name 'train_model' is not defined

In [15]:
model = OurModel()
seed_everything(0)

train = Trainer(max_epochs = 1,
                accelerator = 'gpu',
                devices = -1,
                precision = 16,
                accumulate_grad_batches = 2,
                enable_progress_bar = False,
                num_sanity_val_steps = 0,
                callbacks = [lr_monitor, chekpoint_callback],
                limit_train_batches = 5,
                limit_val_batches = 1
                )

Using cache found in C:\Users\USER1/.cache\torch\hub\facebookresearch_pytorchvideo_main
Seed set to 0
c:\Users\USER1\.virtualenvs\ML-qnch7FHx\Lib\site-packages\lightning_fabric\connector.py:558: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!


MisconfigurationException: No supported gpu backend found!

In [25]:
print(torch.cuda.is_available())

False


In [24]:
from accelerate import Accelerator
import torch


if __name__ == "__main__":
    print("Cuda support:", torch.cuda.is_available(),":", torch.cuda.device_count(), "devices")
    accelerator = Accelerator()
    print(accelerator.state)

Cuda support: False : 0 devices
Distributed environment: DistributedType.NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cpu

Mixed precision type: no

